Credit to:
https://github.com/PGEHackathon/workshop/blob/main/05_machine_learning/workflows/PythonDataBasics_MachineLearning.ipynb, for the random tree

https://www.sciencedirect.com/science/article/pii/S0920410521006343#fig2, for the uncertainty goodness

In [6]:
import os                                                 # to set current working directory
import math                                               # basic calculations like square root
from sklearn.model_selection import train_test_split      # train and test split
from sklearn import tree                                  # tree program from scikit learn (package for machine learning)
from sklearn.metrics import mean_squared_error            # specific measures to check our models
import pandas as pd                                       # DataFrames for tabular data
import numpy as np                                        # arrays and matrix math
import matplotlib.pyplot as plt                           # general plotting
import seaborn as sns                                     # density plots
from sklearn import ensemble
from scipy.stats import norm

#!!!!!!!!!!!!!!!!
import warnings
warnings.filterwarnings("ignore")



# Define custom scores to tune Random Forest

# Scorer1 scores on MSE
def custom_scorer1(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()  # Convert y_true to 1D array
    y_pred = np.asarray(y_pred).ravel()  # Convert y_pred to 1D array
    score = ((y_true - y_pred)**2).mean()
    return score

# Scorer2 scores on Maldonado-Cruz's uncertainty goodness Xi function
def custom_scorer2(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    apd_mean = y_true.mean()
    apd_std = y_true.std()
    xi = np.zeros(16)
    norm_total = 0

    pperc = np.linspace(0.1, 0.8, 16)

    for i in range(16):
        alpha = 1 - pperc[i]  # Convert percentile to alpha
        alpha /= 2  # Divide alpha by 2 for two-tailed confidence interval

        upper_CI = apd_mean + apd_std * norm.ppf(1 - alpha)
        lower_CI = apd_mean - apd_std * norm.ppf(1 - alpha)

        for j in range(0, len(y_pred)):
          if y_pred[j] < upper_CI and y_pred[j] > lower_CI:
            xi[i]=xi[i]+1
        xi[i] = xi[i]/(len(y_true))

        a = 1 if xi[i] > pperc[i] else 0
        norm_total += (((3 * a - 2)) + ((xi[i] - pperc[i]))) * 1 / (len(pperc) + 1)
    return abs(norm_total)



# # Mean imputation, to be replaced
# # Read data
# df = pd.read_csv('/content/sample_data/HackathonData2024.csv')

# # Find all columns with numeric values
# numeric_columns = df.select_dtypes(include=['int64', 'float64'])

# # Replace NaN values with the average of the column
# for column in numeric_columns:
#   df[column] = df[column].fillna(df[column].mean())

# # Save the updated dataframe to a new CSV file
# df.to_csv('/content/sample_data/HackathonData2024_updated.csv', index=False)



my_data = pd.read_csv("/content/HackData_imputed(2).csv")
my_data = my_data.iloc[:,0:12]
numeric_columns = my_data.select_dtypes(include=['int64', 'float64'])

# Define features
f1 = 'PARENT_IN_ZONE_MIN_MAP_DIST'
f2 = 'PARENT_1050_MEDIAN_WELL_AGE'
f3 = 'PARENT_3000_AVG_MAP_DIST'
f4 = 'PARENT_3000_WELL_COUNT'
f5 = 'PARENT_1050_WELL_COUNT'
f6 = 'Soak Time'
f7 = 'PARENT_3000_MEDIAN_WELL_AGE'
X = my_data[[f1, f2, f3, f4, f5, f6, f7]]
y = my_data[['Avg Pump Difference']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None) # train and test split
n_train = len(X_train)
n_test = len(X_test)
print('Number of training ' + str(n_train) + ', number of test ' + str(n_test))




# Instantiate RF
random_forest_reg = ensemble.RandomForestRegressor()

# Train on training data
random_forest_reg.fit(X_train.values, y_train.values)

# Predict using training data set
y_train_pred = random_forest_reg.predict(X_train.values)

# Report the goodness of fit with MSE
MSE_train = mean_squared_error(y_train, y_train_pred)



print('Variance explained training: %.2f' % MSE_train)

y_train_resid = y_train_pred - y_train.values     # calculate the residuals over the training data

print('Training: Average error = %.2f' % np.average(y_train_resid)) # calculate the average testing error
print('Training: Standard Deviation error = %.2f' % np.std(y_train_resid)) # calculate the standard deviation testing error



# Predict using testing data
y_test_pred = random_forest_reg.predict(X_test.values)

# Report the goodness of fit with MSE
MSE_test = mean_squared_error(y_test, y_test_pred)
print('Variance explained testing: %.2f' % MSE_test)

y_test_resid = y_test_pred - y_test.values     # calculate the residuals over the training data

print('Testing: Average error = %.2f' % np.average(y_test_resid)) # calculate the average testing error
print('Testing: Standard Deviation error = %.2f' % np.std(y_test_resid)) # calculate the standard deviation testing error



# Tuning section
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import make_scorer

# Create another RandomForestRegressor
rf_regressor = RandomForestRegressor()

# Define hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(10, 100),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'bootstrap': [True, False]
}

# Create RandomizedSearchCV using custom scorers
custom_score1 = make_scorer(custom_scorer1, greater_is_better=False)
custom_score2 = make_scorer(custom_scorer2, greater_is_better=False)
scorers = {'custom_score1': custom_score1, 'custom_score2': custom_score2}

random_search = RandomizedSearchCV(
    rf_regressor, param_distributions=param_dist, n_iter=10, cv=5, scoring=custom_score2, random_state=42
)

# Fit the RandomizedSearchCV to training data
random_search.fit(X_train, y_train)

# Get and print the best parameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Access the best model
best_rf_model = random_search.best_estimator_

# Now, you can use the best_rf_model for predictions on test data
predictions = best_rf_model.predict(X_test)


# Instantiate the Model with the best parameters
random_forest_reg_tuned = random_search.best_estimator_

# Fit the Data on Training Data
random_forest_reg_tuned.fit(X_train.values, y_train.values)

# Make predictions using the training dataset
y_train_pred = random_forest_reg_tuned.predict(X_train.values)

# Report the goodness of fit
MSE_train_tuned = mean_squared_error(y_train, y_train_pred)     # calculate the training MSE
print('Variance explained training: %.2f' % MSE_train)

y_train_resid = y_train_pred - y_train.values     # calculate the residuals over the training data

print('Tuned Model Training: Average error = %.2f' % np.average(y_train_resid)) # calculate the average testing error
print('Tuned Model Training: Standard Deviation error = %.2f' % np.std(y_train_resid)) # calculate the standard deviation testing error

# Make predictions using the testing dataset
y_test_pred = random_forest_reg_tuned.predict(X_test.values)    # predict with the trained model at the training data samples

# Report the goodness of fit
MSE_test_tuned = mean_squared_error(y_test, y_test_pred)        # calculate the testing MSE
print('Tuned Model Testing: Variance explained = %.2f' % MSE_test)

y_test_resid = y_test_pred - y_test.values        # calculate the residuals over the testing data

print('Tuned Model Testing: Average error = %.2f' % np.average(y_test_resid)) # calculate the average testing error
print('Tuned Model Testing: Standard Deviation error = %.2f' % np.std(y_test_resid)) # calculate the standard deviation testing error



# Step 1. Instantiate the Model
random_forest_reg_final = random_search.best_estimator_ # make the model object and set the hyperparameters

# Step 2: Train the Tuned Model with All Data
random_forest_reg_final.fit(X.values, y.values) # train (fit) the model with the training data



y_test = np.asarray(y_test).ravel()
y_test_pred = np.asarray(y_test_pred).ravel()
apd_mean = y_test.mean()
apd_std = y_test.std()
xi = np.zeros(16)
norm_total = 0

pperc = np.linspace(0.1, 0.8, 16)

for i in range(16):
    alpha = 1 - pperc[i]  # Convert percentile to alpha
    alpha /= 2  # Divide alpha by 2 for two-tailed confidence interval

    upper_CI = apd_mean + apd_std * norm.ppf(1-alpha)
    lower_CI = apd_mean - apd_std * norm.ppf(1-alpha)

    for j in range(0, len(y_test_pred)):
      if y_test_pred[j] < upper_CI and y_test_pred[j] > lower_CI:
        xi[i]=xi[i]+1
    xi[i] = xi[i]/(len(y_test))

    a = 1 if xi[i] > pperc[i] else 0
    norm_total += (((3 * a - 2)) + ((xi[i] - pperc[i]))) * 1 / (len(pperc) + 1)


plt.axline((0, 0), (1, 1))
plt.xlabel("Percent of distribution within confidence interval")
plt.ylabel("Proportion of training predictions within CI")
plt.title("Proportion of Data Points within CI vs. Confidence Interval Width for Testing Data")
plt.xlim(0,1); plt.ylim(0,1)
plt.scatter(pperc, xi,c='red',alpha=0.2,edgecolor='black')
plt.show()


y_train = np.asarray(y_train).ravel()
y_train_pred = np.asarray(y_train_pred).ravel()
apd_mean = y_train.mean()
apd_std = y_train.std()
xi = np.zeros(16)
norm_total = 0

pperc = np.linspace(0.1, 0.8, 16)

for i in range(16):
    alpha = 1 - pperc[i]  # Convert percentile to alpha
    alpha /= 2  # Divide alpha by 2 for two-tailed confidence interval

    upper_CI = apd_mean + apd_std * norm.ppf(1-alpha)
    lower_CI = apd_mean - apd_std * norm.ppf(1-alpha)

    for j in range(0, len(y_train_pred)):
      if y_train_pred[j] < upper_CI and y_train_pred[j] > lower_CI:
        xi[i]=xi[i]+1
    xi[i] = xi[i]/(len(y_train))

    a = 1 if xi[i] > pperc[i] else 0
    norm_total += (((3 * a - 2)) + ((xi[i] - pperc[i]))) * 1 / (len(pperc) + 1)


plt.axline((0, 0), (1, 1))
plt.xlabel("Percent of distribution within confidence interval")
plt.ylabel("Proportion of training predictions within CI")
plt.title("Proportion of Data Points within CI vs. Confidence Interval Width for Training Data")

plt.xlim(0,1); plt.ylim(0,1)
plt.scatter(pperc, xi,c='red',alpha=0.2,edgecolor='black')
plt.show()





# solution = pd.read_csv("/content/solution.csv")
# my_data2 = pd.read_csv("/content/HackData_imputed (1).csv")
# well_ids = [4, 31, 42, 52, 71, 76, 96, 131, 137, 194, 220, 236, 265, 321, 345]

# for j in range(len(well_ids)):
#     random_forest_reg_final = random_search.best_estimator_ # make the model object and set the hyperparameters
#     random_forest_reg_final.fit(X.values, y.values) # train (fit) the model with the training data
#     for i in range(101):
#         X_test2 = my_data2.loc[well_ids[j] - 1, [f1, f2, f3, f4, f5, f6, f7]].values.reshape(1, -1)
#         y_pred = random_forest_reg_final.predict(X_test2)
#         solution.iloc[j, i+1] = y_pred[0]
#         random_forest_reg_final = RandomForestRegressor(n_estimators = i+1)
#         random_forest_reg_final.fit(X.values, y.values)

# solution.to_csv('solution.csv', index=False)



Number of training 266, number of test 67
Variance explained training: 81.26
Training: Average error = 0.53
Training: Standard Deviation error = 29.90
Variance explained testing: 610.64
Testing: Average error = 2.63
Testing: Standard Deviation error = 27.56


KeyboardInterrupt: 

In [ ]:
# prompt: fill cell with given row and column indices

df.loc[row_num, col_num] = value
